# Jupyter Example Notebook

### Check if the kernel is up and working

In [ ]:
print("Kernel works!")

### Perform basic operations to test installed packages

In [ ]:
import torch
# Print PyTorch version
print(f"PyTorch version: {torch.__version__}")

# Check if GPU is available
gpu_available = torch.cuda.is_available()

if gpu_available:
    print("GPUs are available!")
    print("Amount of available GPUs:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i} name:", torch.cuda.get_device_name(i))
else:
    print("GPU is not available.")